# Creating a fiber and running a simulation
This tutorial goes over the basics of using wmglab-neuron to create and simulate the response of a NEURON model of a peripheral nerve axon ("fiber") in response to a time-varying stimulation waveform.

## Creating a fiber
In this tutorial we create a mammalian myelinated fiber model (i.e., MRG) with diameter of 10 µm using an instance of the `FiberModel` class.
We can control the length of the fiber by either specifying the total number of sections (in which case the length will be calculated based on the ultrastructure of the MRG model), or the length of the fiber in µm (in which case the number of sections will be calculated based on the ultrastructure of the MRG model).


See our documentation for more information on how to define other fiber models.

In [ ]:
from wmglab_neuron import build_fiber, FiberModel

n_sections = 265
fiber = build_fiber(FiberModel.MRG_INTERPOLATION, diameter=10, n_sections=n_sections)

This produces a fiber object that can be used to run simulations.

In [ ]:
print(fiber)

## Magic Methods
TODO

In [ ]:
# TODO

## Simulation setup

Before we can run a simulation, we need to create a stimulation waveform (i.e., I(t), the time-course of the extracellular stimulation). We use a biphasic rectangular pulse in this tutorial.

See documentation on stimulation waveforms for more information on creating different waveforms.

In [ ]:
import numpy as np

waveform = np.concatenate((np.ones(100), -np.ones(100), np.zeros(49800)))

Plot the waveform to see what it looks like.

In [ ]:
import matplotlib.pyplot as plt

plt.plot(waveform[:1000])
plt.title('Stimulation waveform')
plt.xlabel('Time step index')
plt.ylabel('Normalized Amplitude')
plt.show()

We also need extracellular potentials along the length of the fiber. We will use an extracellular point source for this tutorial, 250 microns from the fiber, positioned over its center. Our fiber has 265 sections, so we need 265 potentials (i.e., one for the middle of each section).

See our documentation on extracellular potentials for more information.

In [ ]:
fiber.potentials = fiber.point_source_potentials(0, 250, fiber.length / 2, 1, 0.01)

plt.plot(fiber.coordinates, fiber.potentials)
plt.xlabel('Distance along fiber (μm)')
plt.ylabel('Electrical potential (mV)')
plt.title('Extracellular potentials')
plt.show()

## Running a simulation

Specify our simulation parameters as constants.

In [ ]:
time_step = 0.001  # milliseconds
time_stop = 50  # milliseconds

To run a simulation, we need an instance of the Stimulation class. This class is used to run simulations of a fiber's response to a stimulation waveform.

In [ ]:
from wmglab_neuron import ScaledStim

# Create instance of Stimulation class
stimulation = ScaledStim(waveform=waveform, dt=time_step, tstop=time_stop)
print(stimulation)

We can use the `run_sim()` method of the ScaledStim class to run a simulation. By default, this method monitors for action potentials at the distal end of the fiber. `run_sim()` returns the number of detected action potentials and the time of the last action potential.

In [ ]:
# TODO: details about scaling by unit stimulus (1mA) and polarity
stimamp = -1.5  # mA
ap, time = stimulation.run_sim(stimamp, fiber)
print(f'Number of action potentials detected: {ap}')
print(f'Time of last action potential detection: {time} ms')

See also the tutorial for [analyzing results](3_analysis.ipynb).